In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys,cv2,shutil,random
import tensorflow as tf
from tensorflow import keras

from PIL import Image              #create image from array
from sklearn import linear_model   #logistic regression model
from datetime import datetime      #date for submission file
from scipy import ndimage, signal,stats  #image (ndarray) rotation and correlation #stats for mode
from sklearn.preprocessing import PolynomialFeatures # efficient polynomial augmentation
from skimage import img_as_float


#Personal libraries
from proj2_helpers import *
from mask_to_submission import *
from submission_to_mask import *

%load_ext autoreload
%autoreload 2

In [2]:
def LoadTrainingData(n_img, rootdir="Datasets/training/", printnames=False):
    """ Load the data from the root directory. (a total of n_img images) """

    image_dir = rootdir + "images/"
    files = os.listdir(image_dir)

    n = min(n_img, len(files)) # Load maximum 20 images
    print("Loading " + str(n) + " train images...")
    imgs = [load_image(image_dir + files[i]) for i in range(n)]

    gt_dir = rootdir + "groundtruth/"
    print("Loading " + str(n) + " groundtruth images...")
    gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

    if (printnames):
        print("The loaded images are: ")
        for i in range(n):
            print("    - " + files[i])
    
    return imgs, gt_imgs



In [3]:
rootdir="Datasets/training/"
imagedir = rootdir + "images/"
files = os.listdir(imagedir)
n=100
imgs_raw, gt_imgs = LoadTrainingData(n, rootdir)

Loading 100 train images...
Loading 100 groundtruth images...


In [4]:
image_size = 400;
epochs = 5;
x_train = keras.utils.normalize(imgs_raw)

y_train = keras.utils.to_categorical(gt_imgs,num_classes=1)

print("x_train shape:",x_train.shape)
print("y_train shape:", y_train.shape)


x_train shape: (100, 400, 400, 3)
y_train shape: (100, 400, 400, 1)


In [5]:

def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def UNet():
    f = [64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0])
    #c2, p2 = down_block(p1, f[1]) 
    c3, p3 = down_block(p1, f[1]) 
    
    bn = bottleneck(p3, f[2])
    
    u1 = up_block(bn, c3, f[1]) #16 -> 32
    #u2 = up_block(u1, c2, f[1]) #32 -> 64
    u3 = up_block(u1, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u3)
    model = keras.models.Model(inputs, outputs)
    return model


In [6]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 400, 400, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 400, 400, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 400, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 200, 200, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
model.fit(x_train, y_train, batch_size = 32, validation_split = 0.1)

Train on 90 samples, validate on 10 samples
